In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# ---- Create initial population

pm_ecc = holo.population.PM_Eccentricity([0.2,0.2])
pop = holo.population.Pop_Illustris(mods=pm_ecc)

# ---- Apply population modifiers
redz = cosmo.a_to_z(pop.scafa)

hards_no_sesana = [
    #holo.evolution.Hard_GW,
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
]

hards_sesana = [
    #holo.evolution.Hard_GW,
    holo.evolution.Sesana_Scattering(),
    holo.evolution.Dynamical_Friction_NFW(attenuate=False),
    #holo.evolution.CBD_Torques()
]

f_edd = 0.10
eccen = 0

#acc_instance = holo.accretion.Accretion(accmod='Siwek22', f_edd = f_edd, eccen=eccen, subpc=True)

keys = ['no_sesana', 'sesana']
evol_dict = {}
for key,hards in zip(keys,[hards_no_sesana, hards_sesana]):
    evo = holo.evolution.Evolution(pop, hards, nsteps = 100, debug=True, acc=None)
    evo.evolve()
    evol_dict[key] = evo



In [ ]:
labels = [r'$\dot{e}_{\rm b,sesana} = 0$', r'$\dot{e}_{\rm b} \, \rm{Sesana+06}$']

#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 30
xlims = [1.e-2, 1.e1]

n_mbhb = evo.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []#[3,1203]

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
    inds_t_pos = evo.tlook[x] > 0
    if len(evo.tlook[x][inds_t_pos]) > 10:
        print("x = ", x)
        inds_random_mbhb.append(x)
        p += 1

inds_random_mbhb = [1811, 83, 1226]
n_mbhb_to_plot = len(inds_random_mbhb)

fig, axs = plt.subplots(int(n_mbhb_to_plot),3,figsize=(35,15), sharex=True)
fig.suptitle('Effect of Sesana scattering on eccentricity', fontsize=fs)
k=0
for i in range(0,int(n_mbhb_to_plot)):
    for key,label in zip(keys,labels):
        evo = evol_dict[key]
        inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
        time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
        masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
        separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
        dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
        eccen = evo.eccen[inds_random_mbhb[k]][inds_t_pos]
        dedt = evo.dedt[inds_random_mbhb[k]][inds_t_pos]
        inds_lower = separation >= xlims[0]*PC
        inds_upper = separation <= xlims[1]*PC
        inds = inds_lower & inds_upper
        """ CALCULATE HARD RADIUS """
        ses = holo.evolution.Sesana_Scattering()
        total_masses = np.sum(masses,axis=1)[-1]
        vdisp = ses._msigma.vdisp_from_mbh(total_masses, scatter=False)
        mass_ratio_test = masses[:, 1]/masses[:, 0] 
        #index where mass[:, 1] > mass[:, 0], so mass[:, 0] is secondary
        inds_mrat_1 = mass_ratio_test>1
        secondary_mass = np.zeros(np.shape(masses[:, 1]))
        secondary_mass[inds_mrat_1] = masses[:, 0][inds_mrat_1]
        secondary_mass[~inds_mrat_1]  = masses[:, 1][~inds_mrat_1]
        from holodeck.evolution import _Quinlan1996
        rhards = _Quinlan1996.radius_hardening(secondary_mass[-1], vdisp)/PC
        j = 0
        axs[i][j].loglog(separation[inds]/PC, eccen[inds], '*-', label=label)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        axs[i][j].set_ylabel(r'$e_{\rm{b}}$', fontsize=fs)
        #PLOT MASS RATIO
        j = 1
        if masses.T[0][0] >= masses.T[1][0]:
            m1 = masses.T[0]
            m2 = masses.T[1]
        else:
            m1 = masses.T[1]
            m2 = masses.T[0]
        q_b = np.array(m2)/np.array(m1)
        axs[i][j].semilogx(separation[inds]/PC, dedt[inds], '*-', label=label)
        axs[i][j].set_ylabel(r'$\dot{e}_{\rm{b}}$', fontsize=fs)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        j = 2
        axs[i][j].loglog(separation[inds]/PC, dadt[inds], '*-', label=label)
        axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
        axs[i][j].set_ylabel(r'$\dot{a}_{\rm{b}}$', fontsize=fs)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        for j in [0,1,2]:
            #axs[i][j].invert_xaxis()
            if j == 2:
                axs[i][j].legend(fontsize=fs-5)
            axs[i][j].set_xlim(xlims)
            plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
            plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
            ticksize = 25
            tickwidth = 4
            axs[i][j].tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
            axs[i][j].tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
            if i == n_mbhb_to_plot-1:
                axs[i][j].set_xlabel("separation [PC]", fontsize=fs)

    k+=1
    plt.tight_layout()
plt.tight_layout()
plt.savefig("Sesana_Scattering_Eccentricit_Evolution.png", facecolor='w')
    

In [ ]:
labels = [r'$\dot{e}_{\rm b,sesana} = 0$', r'$\dot{e}_{\rm b} \, \rm{Sesana+06}$']

#Take a few random numbers and plot their mass evolutions as a function of time
import random
fs = 30
xlims = [1.e-2, 1.e1]

n_mbhb = evo.shape[0]

n_mbhb_to_plot = 2
inds_random_mbhb = []#[3,1203]

p = 0
while p < n_mbhb_to_plot:
    x = random.randint(0,n_mbhb)
    inds_t_pos = evo.tlook[x] > 0
    if len(evo.tlook[x][inds_t_pos]) > 10:
        print("x = ", x)
        inds_random_mbhb.append(x)
        p += 1

inds_random_mbhb = [1811, 83, 1226]
n_mbhb_to_plot = len(inds_random_mbhb)

fig, axs = plt.subplots(int(n_mbhb_to_plot),3,figsize=(35,15), sharex=True)
fig.suptitle('Effect of Sesana scattering on eccentricity', fontsize=fs)
k=0
for i in range(0,int(n_mbhb_to_plot)):
    for key,label in zip(keys,labels):
        evo = evol_dict[key]
        inds_t_pos = evo.tlook[inds_random_mbhb[k]] > 0
        time = (evo.tlook[inds_random_mbhb[k]][inds_t_pos])
        masses = (evo.mass[inds_random_mbhb[k]][inds_t_pos])
        separation = (evo.sepa[inds_random_mbhb[k]][inds_t_pos])
        dadt = -(evo.dadt[inds_random_mbhb[k]][inds_t_pos])
        eccen = evo.eccen[inds_random_mbhb[k]][inds_t_pos]
        dedt = evo.dedt[inds_random_mbhb[k]][inds_t_pos]
        inds_lower = separation >= xlims[0]*PC
        inds_upper = separation <= xlims[1]*PC
        inds = inds_lower & inds_upper
        """ CALCULATE HARD RADIUS """
        ses = holo.evolution.Sesana_Scattering()
        total_masses = np.sum(masses,axis=1)[-1]
        vdisp = ses._msigma.vdisp_from_mbh(total_masses, scatter=False)
        mass_ratio_test = masses[:, 1]/masses[:, 0] 
        #index where mass[:, 1] > mass[:, 0], so mass[:, 0] is secondary
        inds_mrat_1 = mass_ratio_test>1
        secondary_mass = np.zeros(np.shape(masses[:, 1]))
        secondary_mass[inds_mrat_1] = masses[:, 0][inds_mrat_1]
        secondary_mass[~inds_mrat_1]  = masses[:, 1][~inds_mrat_1]
        from holodeck.evolution import _Quinlan1996
        rhards = _Quinlan1996.radius_hardening(secondary_mass[-1], vdisp)/PC
        j = 0
        axs[i][j].loglog(separation[inds]/PC, eccen[inds], '*-', label=label)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        axs[i][j].set_ylabel(r'$e_{\rm{b}}$', fontsize=fs)
        #PLOT MASS RATIO
        j = 1
        if masses.T[0][0] >= masses.T[1][0]:
            m1 = masses.T[0]
            m2 = masses.T[1]
        else:
            m1 = masses.T[1]
            m2 = masses.T[0]
        q_b = np.array(m2)/np.array(m1)
        axs[i][j].loglog(separation[inds]/PC, dedt[inds], '*-', label=label)
        axs[i][j].set_ylabel(r'$\dot{e}_{\rm{b}}$', fontsize=fs)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        j = 2
        axs[i][j].loglog(separation[inds]/PC, dadt[inds], '*-', label=label)
        axs[i][j].set_xlabel("separation [PC]", fontsize=fs)
        axs[i][j].set_ylabel(r'$\dot{a}_{\rm{b}}$', fontsize=fs)
        if key == keys[-1]:
            axs[i][j].axvline(rhards, color='k', linestyle='--', label = r'$a_{\rm hard}$')
        for j in [0,1,2]:
            #axs[i][j].invert_xaxis()
            if j == 2:
                axs[i][j].legend(fontsize=fs-5)
            axs[i][j].set_xlim(xlims)
            plt.setp(axs[i][j].get_xticklabels(which='both'), fontsize=fs, rotation=45)
            plt.setp(axs[i][j].get_yticklabels(which='both'), fontsize=fs)
            ticksize = 25
            tickwidth = 4
            axs[i][j].tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
            axs[i][j].tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
            if i == n_mbhb_to_plot-1:
                axs[i][j].set_xlabel("separation [PC]", fontsize=fs)

    k+=1
    plt.tight_layout()
plt.tight_layout()
plt.savefig("Sesana_Scattering_Eccentricit_Evolution.png", facecolor='w')
    